In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=9a1795d306340cfa0a8e6cc328369eba1a55f0833231d870366f119c50a06c87
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
conf = SparkConf().setAppName('colab').setMaster("local[*]")
sc = SparkContext(conf=conf)

Task 1

In [4]:
# Create a list containing numbers from 1 to 10
my_list = list(range(1, 11))

# Print the original list
print("Original List:", my_list)

# Square each item in my_list using map() and lambda function
squared_list = list(map(lambda x: x**2, my_list))

# Print the result of squaring each item in the list
print("Squared List:", squared_list)

Original List: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Squared List: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [12]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, count, substring

# Set up SparkConf
conf = SparkConf().setAppName('colab').setMaster("local[*]")


# Initialize SparkSession
spark = SparkSession(sc)

# Task 3

# Load the file
user_comments_rdd = sc.textFile("user_comments.csv")

# create key-value pairs with UserName as key and Comment as value
user_comments_kv = user_comments_rdd.map(lambda line: line.split(",")).map(lambda x: (x[0], x[1]))

# comments given by each user
comments_by_user = user_comments_kv.groupByKey().mapValues(len)

# number of long comments
long_comments_by_user = user_comments_kv.filter(lambda x: len(x[1]) > 20).groupByKey().mapValues(len)

# Count the number of UserNames
usernames_starting_with_alphabet = user_comments_kv.map(lambda x: (x[0][0], 1)).reduceByKey(lambda a, b: a + b)

# maximum number of comments
max_comments_user = comments_by_user.max(lambda x: x[1])

print("Comments by each user:")
print(comments_by_user.collect())

print("Number of long comments by each user:")
print(long_comments_by_user.collect())

print("Usernames starting with each English alphabet:")
print(usernames_starting_with_alphabet.collect())

print("User with maximum number of comments:", max_comments_user)

# Task 4 (
stop_words_rdd = sc.textFile("stop_words.txt")

# Broadcast stop words RDD for efficient use across all nodes
stop_words_broadcast = sc.broadcast(stop_words_rdd.collect())

# Remove stop words from comments
cleaned_comments_rdd = user_comments_rdd.filter(lambda comment: not any(stop_word in comment for stop_word in stop_words_broadcast.value))

# Show cleaned comments
print("Cleaned comments:")
cleaned_comments_rdd.collect()

# Stop SparkContext
sc.stop()


Comments by each user:
[('Aliya153', 1), ('Ali45', 2), ('Sara2', 1)]
Number of long comments by each user:
[('Aliya153', 1), ('Ali45', 1), ('Sara2', 1)]
Usernames starting with each English alphabet:
[('S', 1), ('A', 3)]
User with maximum number of comments: ('Ali45', 2)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/content/stop_words.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.SparkContext.runJob(Spark